In [16]:

Schema from:
* Modules
* Functions
* Classes

1. Get the annotations
2. Get the defaults
3. Make the commands.
4. Create the function.

    Prompt from command.
    Command from schema, module, signature

In [129]:
    import inspect

    import types

    
    import dataclasses, typing, click, inspect, datetime, uuid, pathlib, builtins, sys, contextlib, stringcase

    import inspect

    import types

    
    import dataclasses, typing, click, inspect, datetime, uuid, pathlib, builtins, sys, contextlib, stringcase

In [130]:
    
    def istype(x: typing.Any, y: type) -> bool:
        if isinstance(x, type):
            return issubclass(x, y)
        return False


    def click_type(
        object: typing.Union[type, tuple], default=None
    ) -> typing.Union[type, click.types.ParamType]:
        """Map different python types and objects to click's subset of types."""
        if isinstance(object, typing._GenericAlias):
            return click_type(object.__args__[0], default)
        elif isinstance(object, type):
            if issubclass(object, datetime.datetime):
                return click.DateTime()
            if issubclass(object, typing.Tuple):
                return click.Tuple(object.__args__)
            if issubclass(object, uuid.UUID):
                return click.UUID(default)
            if object is list:
                return
            if issubclass(object, set):
                return click.Choice(object)
            if issubclass(object, pathlib.Path):
                return click.Path()
            if object in {builtins.object, typing.Any}:
                return
            return object
        else:
            if isinstance(object, (tuple, range)):
                if isinstance(object, range):
                    object = object.start, object.stop
                if all(isinstance(x, int) for x in object[:2]):
                    return click.IntRange(*object)
                if all(isinstance(x, float) for x in object[:2]):
                    return click.FloatRange(*object)

In [131]:
    
    @dataclasses.dataclass(order=True)
    class Commands:
        """Build click commands from objects."""
        object: object
        def decorators(self) -> typing.List[click.Command]:
            return [
                command.build() 
                if isinstance(command, Command) 
                else command 
                for command in self
            ]
                
        def decorate(self, *callable):
            command = []
            for object in callable:
                for cmd in reversed(self.decorators()):
                    object = cmd(object)
                command.append(click.command()(object))
            if len(command) == 1:
                return command[0]
            group = click.Group()
            for object in command:
                group.add_command(object)
            return group
                            
        def command(self):
            return self.decorate(self.object)


In [132]:
    
    @dataclasses.dataclass
    class Command:
        name: str
        type: type
        default: object
        option: bool
        description: str = None
            
        def build(self):    
            return self._build_option('--'+self.name) if self.option else self._build_argument(self.name)
        
        def _build_option(self, *args, **opts):
            """Build a click option."""
            opts.update(
                default=self.default,
                type=click_type(self.type, self.default),
                multiple=isinstance(self.type, typing.List),
                show_default=True,
                is_flag=self.type is bool,
            )
            self.description is None or opts.update(help=self.description)
            return click.option(*args, **opts)
        def _build_argument(self, *args, **opts):
            opts.update(
                type=click_type(self.type)
            )
            return click.argument(*args, **opts)

In [133]:
    @dataclasses.dataclass
    class Function(Commands):
        def __iter__(self):
            signature = inspect.signature(self.object)
            for k, v in signature.parameters.items():
                if k != 'ctx':
                    yield Command(
                        name=stringcase.spinalcase(k),
                        type=v.annotation,
                        default=None if v.default != inspect._empty else v.default,
                        option=v.default != inspect._empty, 
                    )
            for k, v in signature.parameters.items():
                if k == "ctx": yield click.pass_context
                    
        def command(self):
            return self.decorate(self.object)

    @dataclasses.dataclass
    class Function(Commands):
        def __iter__(self):
            signature = inspect.signature(self.object)
            for k, v in signature.parameters.items():
                if k != 'ctx':
                    yield Command(
                        name=stringcase.spinalcase(k),
                        type=v.annotation,
                        default=None if v.default != inspect._empty else v.default,
                        option=v.default != inspect._empty, 
                    )
            for k, v in signature.parameters.items():
                if k == "ctx": yield click.pass_context
                    
        def command(self):
            return self.decorate(self.object)

In [134]:
    
    class Module(Commands):
        def __iter__(self):
            for k, v in getattr(self.object, '__annotations__'):
                yield Command(
                    name=stringcase.spinalcase(k), type=v, default=getattr(self.object, k, None), option=k in vars(self.object), )

In [135]:
    _schema_mapping = dict(zip('boolean string integer number object array null'.split(), (bool, str, int, float, object, list, None)))

    _schema_mapping = dict(zip('boolean string integer number object array null'.split(), (bool, str, int, float, object, list, None)))

In [136]:
    def _map_schema(object):
        if isinstance(object, str): return _schema_mapping[object]
        else:
            if 'enum' in object:
                return set(object['enum'])

    def _map_schema(object):
        if isinstance(object, str): return _schema_mapping[object]
        else:
            if 'enum' in object:
                return set(object['enum'])

In [137]:
    class Schema(Commands):
        def __iter__(self):
            for k, v in self.object.get('properties', {}):
                yield Command(
                    name=stringcase.spinalcase(k),
                    type=_map_schema(v.get('type', None)),
                    description=v.get('description', None),
                    default=self.object['properties'].get('default', None),
                    option='default' in self.object['properties'], 
                )

    class Schema(Commands):
        def __iter__(self):
            for k, v in self.object.get('properties', {}):
                yield Command(
                    name=stringcase.spinalcase(k),
                    type=_map_schema(v.get('type', None)),
                    description=v.get('description', None),
                    default=self.object['properties'].get('default', None),
                    option='default' in self.object['properties'], 
                )

In [104]:
    @contextlib.contextmanager
    def argv(argv):
        replace = sys.argv
        sys.argv= argv.split()
        yield
        sys.argv = replace

    @contextlib.contextmanager
    def argv(argv):
        replace = sys.argv
        sys.argv= argv.split()
        yield
        sys.argv = replace

In [111]:
    @dataclasses.dataclass
    class CLI:
        object: typing.Any
        command: click.Command = dataclasses.dataclass(init=False)
        def __call__(self):
            try:return self.command.main()
            except SystemExit:...
        def __post_init__(self):
            commands = []
            if not isinstance(self.object, typing.Iterable):
                self.object = self.object,
            for thing in self.object:
                if inspect.isfunction(thing):
                    commands.append(Function(object=thing).command())
                if isinstance(thing, types.ModuleType):
                    commands.append(Module(object=thing).command())
                if isinstance(thing, dict):
                    commands.append(Schema(object=thing).command())
            if len(commands) == 1:
                self.command = commands[0]
            else:
                self.command = click.Group()
                for thing in commands:
                    self.command.add_command(thing)

    @dataclasses.dataclass
    class CLI:
        object: typing.Any
        command: click.Command = dataclasses.dataclass(init=False)
        def __call__(self):
            try:return self.command.main()
            except SystemExit:...
        def __post_init__(self):
            commands = []
            if not isinstance(self.object, typing.Iterable):
                self.object = self.object,
            for thing in self.object:
                if inspect.isfunction(thing):
                    commands.append(Function(object=thing).command())
                if isinstance(thing, types.ModuleType):
                    commands.append(Module(object=thing).command())
                if isinstance(thing, dict):
                    commands.append(Schema(object=thing).command())
            if len(commands) == 1:
                self.command = commands[0]
            else:
                self.command = click.Group()
                for thing in commands:
                    self.command.add_command(thing)

In [128]:
    __test__ = {"function": """
    >>> def f(i: int=2): return i

    >>> def g(i: int=2): return i

    >>> with argv('test f --help'): CLI((f, g))()
    Usage: test f [OPTIONS]
    <BLANKLINE>
    Options:
      --i INTEGER
      --help       Show this message and exit.

    >>> with argv('test g --help'): CLI((f, g))()
    Usage: test g [OPTIONS]
    <BLANKLINE>
    Options:
      --i INTEGER
      --help       Show this message and exit.

    >>> with argv('test --help'): CLI((f, g))()
    Usage: test [OPTIONS] COMMAND [ARGS]...
    <BLANKLINE>
    Options:
      --help  Show this message and exit.
    <BLANKLINE>
    Commands:
      f
      g
      
    """}

    __test__ = {"function": """
    >>> def f(i: int=2): return i

    >>> def g(i: int=2): return i

    >>> with argv('test f --help'): CLI((f, g))()
    Usage: test f [OPTIONS]
    <BLANKLINE>
    Options:
      --i INTEGER
      --help       Show this message and exit.

    >>> with argv('test g --help'): CLI((f, g))()
    Usage: test g [OPTIONS]
    <BLANKLINE>
    Options:
      --i INTEGER
      --help       Show this message and exit.

    >>> with argv('test --help'): CLI((f, g))()
    Usage: test [OPTIONS] COMMAND [ARGS]...
    <BLANKLINE>
    Options:
      --help  Show this message and exit.
    <BLANKLINE>
    Commands:
      f
      g
      
    """}

In [ ]:
    import inspect, click, stringcase, enum, uuid, datetime, pathlib, typing, types, builtins, copy, jsonschema


    def autoclick(
        *object: typing.Union[types.FunctionType, click.Command], group=None, **settings
    ) -> click.Command:
        """Automatically generate a click command line application using type inference."""
        app = group or click.Group()
        for command in object:
            if isinstance(command, click.Command):
                app.add_command(command)
            elif isinstance(command, types.FunctionType):
                decorators = signature_to_decorators(command)
                command = command_from_decorators(
                    command, *decorators, **settings, help=inspect.getdoc(command)
                )
                app.add_command(command)
            elif isinstance(command, dict):
                try:
                    jsonschema.validate(command, jsonschema.Draft7Validator.META_SCHEMA)
                    decorators = decorators_from_schema(command)
                except: decorators = decorators_from_dict(command)
                command = command_from_decorators(None, *decorators, **settings)
        return command if len(object) == 1 else app


    def istype(x: typing.Any, y: type) -> bool:
        if isinstance(x, type):
            return issubclass(x, y)
        return False


    def click_type(
        object: typing.Union[type, tuple], default=None
    ) -> typing.Union[type, click.types.ParamType]:
        """Translate python types to click's subset of types."""
        if isinstance(object, typing._GenericAlias):
            return click_type(object.__args__[0], default)
        elif isinstance(object, type):
            if issubclass(object, datetime.datetime):
                return click.DateTime()
            if issubclass(object, typing.Tuple):
                return click.Tuple(object.__args__)
            if issubclass(object, uuid.UUID):
                return click.UUID(default)
            if object is list:
                return
            if issubclass(object, set):
                return click.Choice(object)

            if issubclass(object, pathlib.Path):
                return click.Path()
            if object in {builtins.object, typing.Any}:
                return
            return object
        else:
            if isinstance(object, tuple):
                if all(isinstance(x, int) for x in object[:2]):
                    return click.IntRange(*object)
                if all(isinstance(x, float) for x in object[:2]):
                    return click.FloatRange(*object)


    def command_from_decorators(command, *decorators, **settings):
        if command is None:
            *decorators, command = decorators
        for decorator in reversed(decorators):
            command = decorator(command)
        return click.command(no_args_is_help=bool(decorators), **settings)(command)


    def decorators_from_dicts(annotations, defaults, *decorators):
        """This is the last place we go.  Dicts are the general way to make commands.
        annotations define what becomes a type."""
        for k, v in annotations.items():
            if k in defaults:
                t = click_type(v, defaults.get(k))
                decorators += (
                    click.option(
                        "-" * (1 if len(k) == 1 else 2) + stringcase.spinalcase(k),
                        type=t,
                        default=defaults.get(k),
                        show_default=True,
                        is_flag=v is bool,
                    ),
                )

            elif isinstance(v, typing._GenericAlias) or istype(v, list):
                decorators += (
                    click.argument(
                        stringcase.spinalcase(k),
                        type=click_type(getattr(v, "__args__", (str,))[0]),
                        nargs=-1,
                    ),
                )
            else:
                decorators += (
                    click.argument(stringcase.spinalcase(k), type=click_type(v)),
                )
        return decorators


    def decorators_from_dict(object):
        return decorators_from_dicts(object.get("__annotations__", {}), object)


    def decorators_from_module(object):
        return decorators_from_dict(vars(object))


    def signature_to_decorators(object, *decorators):
        signature = inspect.signature(object)
        decorators += decorators_from_dicts(
            {
                k: typing.List[v.annotation]
                if v.kind == inspect._ParameterKind.VAR_POSITIONAL
                else v.annotation
                for k, v in signature.parameters.items()
                if k != "ctx"
            },
            {
                k: v.default
                for k, v in signature.parameters.items()
                if v.default != inspect._empty
            },
        )
        for k, v in signature.parameters.items():
            if k == "ctx":
                decorators += (click.pass_context,)
            break
        return decorators


In [ ]:
    if __name__ == '__main__': 
        if '__file__' in locals():
            if 'covtest' in __import__('sys').argv:
                print(__import__('doctest').testmod(optionflags=8))
        else:
            import IPython; complement, copy, compose
            !jupyter nbconvert --to python --TemplateExporter.exclude_input_prompt=True cleye.ipynb
            with IPython.utils.capture.capture_output():
                !black cleye.py
            !isort cleye.py
            !ipython -m coverage -- run cleye.py covtest
            !coverage report
            !coverage html
            with IPython.utils.capture.capture_output():
                !pyreverse cleye -osvg -pcleye
            IPython.display.display(IPython.display.SVG('classes_cleye.svg'))
            with IPython.utils.capture.capture_output():
                !pyreverse cleye -osvg -pcleye -my -s1
            IPython.display.display(IPython.display.SVG('classes_cleye.svg'))